In [6]:
from transformers import pipeline, AutoTokenizer, TFAutoModelForSequenceClassification, AutoConfig
import torch
import torch.nn.functional as F
import pandas as pd

In [7]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

In [8]:
classifier = pipeline(model = MODEL, tokenizer= tokenizer, config = config )

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
X_train = pd.read_csv('C://Users/yashl/Downloads/Compressed/archive_6/train.csv')
X_test = pd.read_csv('C://Users/yashl/Downloads/Compressed/archive_6/test.csv')

In [10]:
X_train.head()

,2,Stuning even for the non-gamer,This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^
0,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
1,2,Amazing!,This soundtrack is my favorite music of all ti...
2,2,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...
3,2,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine..."
4,2,an absolute masterpiece,I am quite sure any of you actually taking the...


In [11]:
ax = pd.DataFrame(X_train)

In [12]:
ax.rename(columns = {'This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^' : 'review'} , inplace = True)

In [13]:
ax.rename(columns = {'Stuning even for the non-gamer' : 'review-title'} , inplace = True)

In [14]:
ax.rename(columns = {'2' : 'score'} , inplace = True)

In [15]:
## Set labels to 0 and 1
def decrease_label_by_one(label):
    return label - 1

X_train['score'] = X_train['score'].apply(decrease_label_by_one)

In [16]:
print(ax)

         score                                       review-title  \
0            1              The best soundtrack ever to anything.   
1            1                                           Amazing!   
2            1                               Excellent Soundtrack   
3            1  Remember, Pull Your Jaw Off The Floor After He...   
4            1                            an absolute masterpiece   
...        ...                                                ...   
3599994      0                                      Don't do it!!   
3599995      0                      Looks nice, low functionality   
3599996      0                         compact, but hard to clean   
3599997      0                                 what is it saying?   
3599998      1              Makes My Blood Run Red-White-And-Blue   

                                                    review  
0        I'm reading a lot of reviews saying that this ...  
1        This soundtrack is my favorite music of 

In [19]:
batch= tokenizer(X_train['review'], padding=True, truncation=True, max_length=512, return_tensors="pt")
print(batch)

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [20]:
import numpy as np
import pandas as pd

In [21]:
reviews = ax['review']

In [22]:
print(reviews)

0          I'm reading a lot of reviews saying that this ...
1          This soundtrack is my favorite music of all ti...
2          I truly like this soundtrack and I enjoy video...
3          If you've played the game, you know how divine...
4          I am quite sure any of you actually taking the...
                                 ...                        
3599994    The high chair looks great when it first comes...
3599995    I have used this highchair for 2 kids now and ...
3599996    We have a small house, and really wanted two o...
3599997    not sure what this book is supposed to be. It ...
3599998    I agree that every American should read this b...
Name: review, Length: 3599999, dtype: object


In [23]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [26]:
print(df.tail())

                                                    review
3599994  The high chair looks great when it first comes...
3599995  I have used this highchair for 2 kids now and ...
3599996  We have a small house, and really wanted two o...
3599997  not sure what this book is supposed to be. It ...
3599998  I agree that every American should read this b...


In [27]:
df['review'].iloc[0]

"I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny."

In [50]:
def sentiment_score(review):
    res = classifier(review)
    return res[0]['score']

In [51]:
sentiment_score(df['review'].iloc[0])

0.6653041243553162

In [53]:
df_test = df.head(100)

In [ ]:
df['sentiment'] = df['review'].apply(lambda x : sentiment_score(x[:512]))

In [59]:
print(df.head())

                                              review  sentiment
0  I'm reading a lot of reviews saying that this ...   0.665304
1  This soundtrack is my favorite music of all ti...   0.951641
2  I truly like this soundtrack and I enjoy video...   0.978255
3  If you've played the game, you know how divine...   0.979131
4  I am quite sure any of you actually taking the...   0.864379
